In [1]:
import itertools
import shutil
import os
from os import path
from glob import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from IPython.display import display

In [2]:
wfdp_path = '/datasets/rpartsey/satellite/planet/wfdp_labels'

In [3]:
geojson_paths = glob(path.join(wfdp_path, '*', '*.geojson'))
[path.basename(p) for p in geojson_paths]

['wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-16.geojson',
 'wfdp_Zaporozhje_2019_spring_Leiberiuk_2019-06-11.geojson',
 'wfdp_Zaporozhje_2017_autumn_Leiberiuk_2019-02-17.geojson',
 'wfdp_Zaporozhje_2017_spring_Leiberiuk_2019-02-08.geojson',
 'wfdp_Zaporozhje_2018_autumn_Leiberiuk_2019-01-16.geojson',
 'wfdp_Zaporozhje_2018_spring_Leiberiuk_2019-01-16.geojson',
 'wfdp_Lugansk_2017_summer_Leiberiuk_2019-09-13.geojson',
 'wfdp_Lugansk_2017_spring_Leiberiuk_2019-10-08.geojson',
 'wfdp_Lugansk_2018_autumn_Kuzminova_2019-02-22.geojson',
 'wfdp_Lugansk_2018_spring_Kuzminova_2019-03-27.geojson',
 'wfdp_Lugansk_2018_autumn_Kuzminova_2019_04_30.geojson',
 'wfdp_Lugansk_2018_autumn_Kuzminova_2019-03-27.geojson',
 'wfdp_Lugansk_2018_summer_Kuzminova_2019-03-27.geojson',
 'wfdp_Lugansk_2017_autumn_Kuzminova_2019_06_22.geojson',
 'wfdp_Lugansk_2018_summer_Kuzminova_2019_04_30.geojson',
 'wfdp_Lugansk_2018_spring_Kuzminova_2019-06-05.geojson',
 'wfdp_Kharkiv_2017_summer_Leiberiuk_2019-07-02.geojso

In [4]:
dfs = []
for p in geojson_paths:
    df = gpd.read_file(p) 
    df['source'] = path.basename(p)
    dfs.append(df)
    
gdf = pd.concat(dfs).reset_index(drop=True)
gdf.shape

(2277, 6)

In [5]:
gdf.loc[95]

date                               20190313_080517_1040_tms.xml
planet_img                                           2019-03-12
firms_sour                                                 None
editor                               Oleksandr Leiberiuk - 0057
geometry      POLYGON ((36.05376137457734 47.64155603858262,...
source        wfdp_Zaporozhje_2019_spring_Leiberiuk_2019-06-...
Name: 95, dtype: object

In [6]:
# fix incorrect date
date = gdf.loc[95, 'date']
gdf.loc[95, 'date'] = gdf.loc[95, 'planet_img']
gdf.loc[95, 'planet_img'] = date

In [7]:
gdf.loc[95]

date                                                 2019-03-12
planet_img                         20190313_080517_1040_tms.xml
firms_sour                                                 None
editor                               Oleksandr Leiberiuk - 0057
geometry      POLYGON ((36.05376137457734 47.64155603858262,...
source        wfdp_Zaporozhje_2019_spring_Leiberiuk_2019-06-...
Name: 95, dtype: object

In [8]:
gdf.head()

,date,planet_img,firms_sour,editor,geometry,source
0,2018-08-28,20180831_075824_0f35_tms.xml,V1,Oleksandr Leiberiuk - 0001,"POLYGON ((36.62941 47.27804, 36.63887 47.27809...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...
1,2018-08-22,20180823_075453_0e20_tms.xml,V1,Oleksandr Leiberiuk - 0002,"POLYGON ((36.85774 46.82754, 36.85798 46.82746...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...
2,2018-08-23,20180824_075716_1011_tms.xml,V1,Oleksandr Leiberiuk - 0003,"POLYGON ((36.85279 46.81438, 36.85312 46.81543...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...
3,2018-08-22,20180823_073520_0f3c_tms.xml,_,Oleksandr Leiberiuk - 0004,"POLYGON ((35.86952 46.65867, 35.86971 46.65775...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...
4,2018-08-22,20180823_073521_0f3c_tms.xml,_,Oleksandr Leiberiuk - 0005,"POLYGON ((35.85320 46.66909, 35.85242 46.67009...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...


In [9]:
none_fields = gdf.planet_img.isna() | gdf.geometry.isna()
valid_polygons = gdf.geometry.is_valid

gdf = gdf[(~none_fields) & valid_polygons]

symbols = ['Sentinel', '10.04.18', ',', '?????', 'NULL', '(', ')']
invalid_symbols = gdf.planet_img.map(lambda p: any(s in p for s in symbols))

gdf = gdf[~invalid_symbols]

gdf.shape

(2013, 6)

In [10]:
gdf_copy = gdf.copy()
old_epsg = gdf_copy.crs.to_epsg()
old_epsg

4326

In [11]:
gdf_copy = gdf_copy.to_crs(epsg=32636)

In [12]:
gdf_copy['poly_bounds'] = gdf_copy.geometry.map(lambda poly: poly.bounds)

In [13]:
COORD_TO_PIXEL_RATIO = 3 # coord / pixel = 3 / 1 in gdf.to_crs(epsg=32636)
PIXEL_PADDING = 128
COORD_PADDING = PIXEL_PADDING * COORD_TO_PIXEL_RATIO

def expand_bounds(bounds):
    minx, miny, maxx, maxy = bounds
    
    minx -= COORD_PADDING
    miny -= COORD_PADDING
    maxx += COORD_PADDING
    maxy += COORD_PADDING
    
    return minx, miny, maxx, maxy

gdf_copy['expanded_bounds'] = gdf_copy['poly_bounds'].map(expand_bounds)

In [14]:
gdf_copy['bbox'] = gdf_copy.expanded_bounds.map(lambda bounds: box(*bounds))

In [15]:
gdf_copy.head()

,date,planet_img,firms_sour,editor,geometry,source,poly_bounds,expanded_bounds,bbox
0,2018-08-28,20180831_075824_0f35_tms.xml,V1,Oleksandr Leiberiuk - 0001,"POLYGON ((774478.951 5242453.718, 775194.042 5...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"(774478.9507938821, 5241087.532265591, 775385....","(774094.9507938821, 5240703.532265591, 775769....","POLYGON ((775769.9788487663 5240703.532265591,..."
1,2018-08-22,20180823_075453_0e20_tms.xml,V1,Oleksandr Leiberiuk - 0002,"POLYGON ((794215.114 5193228.641, 794233.625 5...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"(793181.5444087437, 5189774.82635592, 796043.9...","(792797.5444087437, 5189390.82635592, 796427.9...","POLYGON ((796427.9504789268 5189390.82635592, ..."
2,2018-08-23,20180824_075716_1011_tms.xml,V1,Oleksandr Leiberiuk - 0003,"POLYGON ((793909.953 5191747.327, 793928.758 5...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"(793833.1318794352, 5191523.895850659, 794612....","(793449.1318794352, 5191139.895850659, 794996....","POLYGON ((794996.6478769487 5191139.895850659,..."
3,2018-08-22,20180823_073520_0f3c_tms.xml,_,Oleksandr Leiberiuk - 0004,"POLYGON ((719537.098 5171234.474, 719555.721 5...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"(719487.5977900957, 5171072.340910594, 719558....","(719103.5977900957, 5170688.340910594, 719942....","POLYGON ((719942.080174937 5170688.340910594, ..."
4,2018-08-22,20180823_073521_0f3c_tms.xml,_,Oleksandr Leiberiuk - 0005,"POLYGON ((718246.626 5172346.288, 718182.996 5...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"(717862.227695063, 5171777.658725349, 718428.0...","(717478.227695063, 5171393.658725349, 718812.0...","POLYGON ((718812.0377264365 5171393.658725349,..."


In [16]:
gdf_copy = gdf_copy.set_geometry('bbox')
gdf_copy = gdf_copy.to_crs(epsg=old_epsg)
gdf_copy = gdf_copy.set_geometry('geometry')

In [17]:
gdf['bbox'] = gdf_copy['bbox'].copy()

In [18]:
gdf.head()

,date,planet_img,firms_sour,editor,geometry,source,bbox
0,2018-08-28,20180831_075824_0f35_tms.xml,V1,Oleksandr Leiberiuk - 0001,"POLYGON ((36.62941 47.27804, 36.63887 47.27809...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"POLYGON ((36.64537 47.26179, 36.64671 47.28129..."
1,2018-08-22,20180823_075453_0e20_tms.xml,V1,Oleksandr Leiberiuk - 0002,"POLYGON ((36.85774 46.82754, 36.85798 46.82746...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"POLYGON ((36.88420 46.79210, 36.88698 46.83070..."
2,2018-08-23,20180824_075716_1011_tms.xml,V1,Oleksandr Leiberiuk - 0003,"POLYGON ((36.85279 46.81438, 36.85312 46.81543...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"POLYGON ((36.86661 46.80844, 36.86771 46.82365..."
3,2018-08-22,20180823_073520_0f3c_tms.xml,_,Oleksandr Leiberiuk - 0004,"POLYGON ((35.86952 46.65867, 35.86971 46.65775...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"POLYGON ((35.87455 46.65363, 35.87499 46.66199..."
4,2018-08-22,20180823_073521_0f3c_tms.xml,_,Oleksandr Leiberiuk - 0005,"POLYGON ((35.85320 46.66909, 35.85242 46.67009...",wfdp_Zaporozhje_2018_summer_Leiberiuk_2019-01-...,"POLYGON ((35.86013 46.66034, 35.86086 46.67407..."


In [19]:
len(gdf.groupby('planet_img'))

1469

In [20]:
def polygons_intersect(poly, other):
    return poly['bbox'].intersects(other['geometry']) or other['bbox'].intersects(poly['geometry'])

def merge_geometries(geometry, other):
    return geometry.union(other)

def merge_bboxes(bbox, other):
    return box(*merge_geometries(bbox, other).bounds)

def merge_sources(src, other):
    return ', '.join(sorted(set(src.split(', ') + other.split(', '))))


processed_polygons = {}

for key, group_df in gdf.groupby('planet_img'):
    group_df = group_df.reset_index(drop=True)
    d = {index: row.to_dict() for index, row in group_df.iterrows()}

    print(key)
    print('Number of elements before merge: ', len(d))
    
    while any(polygons_intersect(d[i], d[j]) for i, j in itertools.combinations(d.keys(), 2)):
        merged = set()
        for i, j in itertools.combinations(d.keys(), 2):
            poly = d[i]
            other = d[j]
            if polygons_intersect(poly, other):
                poly['geometry'] = merge_geometries(poly['geometry'], other['geometry'])
                poly['bbox'] = merge_bboxes(poly['bbox'], other['bbox'])
                poly['source'] = merge_sources(poly['source'], other['source'])
                merged.add(j)
                
        for k in merged:
            d.pop(k)
    
    print('Number of elements after merge: ', len(d), end='\n\n')
    
    processed_polygons.update({key: d.values()})

20170302_081157_0c46_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170304_074225_0e30_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20170305_072940_0f1d_tms.xml
Number of elements before merge:  2
Number of elements after merge:  1

20170305_073625_0e3a_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170305_073646_1029_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170305_075224_0e16_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170307_074636_0e1f_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170308_074229_0e14_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170309_073253_100f_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170309_073254_100f_tms.xml
Number of elements before merge:  1
Number of elements after m

20170326_075341_0e1f_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170328_073754_0e19_tms.xml
Number of elements before merge:  3
Number of elements after merge:  3

20170328_074032_1038_tms.xml
Number of elements before merge:  2
Number of elements after merge:  1

20170328_074602_0e30_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170328_074641_0e30_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170328_074642_0e30_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20170328_074645_0e30_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170328_074737_0f1d_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20170329_072711_0f1d_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170329_072715_0f1d_tms.xml
Number of elements before merge:  3
Number of elements after m

20170403_074631_0e0e_tms.xml
Number of elements before merge:  5
Number of elements after merge:  3

20170403_074632_0e0e_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20170403_074639_0e0e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170403_074644_0e0e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170403_074648_0e0e_tms.xml
Number of elements before merge:  4
Number of elements after merge:  3

20170403_074649_0e0e_tms.xml
Number of elements before merge:  3
Number of elements after merge:  1

20170403_074650_0e0e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170403_074653_0e0e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170403_074706_0e0e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170403_074707_0e0e_tms.xml
Number of elements before merge:  1
Number of elements after m

20170413_073804_1024_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20170413_073806_1024_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170413_073854_1024_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170413_073919_0e0e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170413_073922_0e0e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170413_074707_0e16_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20170413_074730_0e16_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170413_074754_0e16_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170413_111306_0c1b_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170413_111323_0c1b_tms.xml
Number of elements before merge:  1
Number of elements after m

Number of elements after merge:  1

20170722_073158_1039_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170722_073644_101d_tms.xml
Number of elements before merge:  3
Number of elements after merge:  2

20170722_073645_101d_tms.xml
Number of elements before merge:  2
Number of elements after merge:  1

20170725_073409_100c_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20170725_073410_100c_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170725_074437_102a_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170725_074438_102a_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170725_074756_102f_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170726_073328_102d_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20170726_074426_1039_tms.xml
Number of elements before 

...

20180411_073955_0f44_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180411_074534_0f43_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180411_074612_103b_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180411_074630_103b_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180411_074631_103b_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180411_074818_0f17_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180411_075127_0e16_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20180411_075128_0e16_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180411_075402_0f51_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180411_075540_0e26_tms.xml
Number of elements before merge:  3
Number of elements after m

20180530_075034_1053_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180609_080255_0f25_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180610_072921_104d_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180611_074938_1008_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180613_074922_101e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180614_074143_0f21_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180614_074800_0f34_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180614_075227_101b_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180614_075954_1025_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180614_080442_1033_tms.xml
Number of elements before merge:  1
Number of elements after m

Number of elements after merge:  2

20180830_080321_1014_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180831_075824_0f35_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180831_080345_100e_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180901_071913_1054_tms.xml
Number of elements before merge:  4
Number of elements after merge:  3

20180901_075055_1005_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180901_080233_100a_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20180904_075039_1008_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180904_080343_0f35_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20180905_075209_1002_tms.xml
Number of elements before merge:  3
Number of elements after merge:  1

20180905_075742_0e19_tms.xml
Number of elements before 

Number of elements after merge:  1

20181119_075811_0f35_tms.xml
Number of elements before merge:  6
Number of elements after merge:  2

20181123_075809_1009_tms.xml
Number of elements before merge:  3
Number of elements after merge:  1

20181123_075856_100c_tms.xml
Number of elements before merge:  3
Number of elements after merge:  1

20181124_075040_1002_tms.xml
Number of elements before merge:  8
Number of elements after merge:  3

20190214_070744_0f3c_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190218_070439_104a_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190218_080429_1008_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190218_080528_1032_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190219_070437_1043_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190219_080332_100e_tms.xml
Number of elements before 

20190401_080459_101f_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190401_080505_101f_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20190401_080654_1003_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190401_080708_1003_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190401_080916_0e20_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190401_081001_1002_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190401_081026_1002_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190401_081500_1025_tms.xml
Number of elements before merge:  1
Number of elements after merge:  1

20190402_065012_1048_tms.xml
Number of elements before merge:  2
Number of elements after merge:  2

20190402_065306_104b_tms.xml
Number of elements before merge:  2
Number of elements after m

In [21]:
len(processed_polygons)

1469

In [25]:
# generating geojson files

poly_dir = path.join(wfdp_path, 'processed', 'poly')
bbox_dir = path.join(wfdp_path, 'processed', 'bbox')

for planet_img, values in processed_polygons.items():
    image_id = planet_img[:-8]
    ext = planet_img[-8:]
    
    assert ext == '_tms.xml', planet_img

    file_name = '{}.geojson'.format(image_id)
    poly_path = path.join(poly_dir, file_name)
    bbox_path = path.join(bbox_dir, file_name)
        
    poly_gdf = gpd.GeoDataFrame(values)
    
    bbox_gdf = poly_gdf.copy()
    bbox_gdf['geometry'] = bbox_gdf['bbox']
    bbox_gdf = bbox_gdf.set_geometry('geometry')
    
    poly_gdf = poly_gdf.drop('bbox', axis=1)
    poly_gdf.to_file(poly_path, driver='GeoJSON')
    
    bbox_gdf = bbox_gdf.drop('bbox', axis=1)
    bbox_gdf.to_file(bbox_path, driver='GeoJSON')    
#     if len(values) > 1:
#         print(values)
#         foo = gpd.GeoDataFrame(values)
#         display(gpd.GeoDataFrame(values))
#         break
#     for index, v in enumerate(values):
#         poly = v['geometry']
#         bbox = v['bbox']
        
#         image_id = k[:-8]
#         ext = image_name[-8:]
#         assert ext == '_tms.xml'

#         file_name = '{}-{}.geojson'.format(image_id, index)
#         poly_path = os.path.join(poly_dir, file_name)
#         bbox_path = os.path.join(bbox_dir, file_name)
        
#         save_shape(poly, gdf.crs, poly_path)
#         save_shape(bbox, gdf.crs, bbox_path)

In [26]:
bbox_gdf

,date,planet_img,firms_sour,editor,geometry,source
0,2019-04-25,20190503_081408_1032_tms.xml,_,Oleksandr Leiberiuk - 0038,"POLYGON ((35.08720 49.16211, 35.08754 49.17027...",wfdp_Kharkiv_2019_spring_Leiberiuk_2019-08-25....


In [27]:
poly_gdf

,date,planet_img,firms_sour,editor,geometry,source
0,2019-04-25,20190503_081408_1032_tms.xml,_,Oleksandr Leiberiuk - 0038,"POLYGON ((35.08096 49.16646, 35.08120 49.16665...",wfdp_Kharkiv_2019_spring_Leiberiuk_2019-08-25....


In [28]:
pd.merge(poly_gdf, bbox_gdf[['geometry']], how='inner', left_index=True, right_index=True, suffixes=('_poly', '_bbox'))

,date,planet_img,firms_sour,editor,geometry_poly,source,geometry_bbox
0,2019-04-25,20190503_081408_1032_tms.xml,_,Oleksandr Leiberiuk - 0038,"POLYGON ((35.08096 49.16646, 35.08120 49.16665...",wfdp_Kharkiv_2019_spring_Leiberiuk_2019-08-25....,"POLYGON ((35.08720 49.16211, 35.08754 49.17027..."
